# 实现LeNet-5模型对CIFAR-10数据集进行分类

## LeNet-5模型

LeNet-5模型是Yan Lecun在1998年提出的一种卷积神经网络模型，最初设计用于手写数字的识别，是早期卷积神经网络中最有代表性的实验系统之一。[论文原地址链接](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf)。以下是该模型的结构。![lennet](https://img-blog.csdn.net/20171018154341615?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvaGFwcHlvcmc=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

## CIFAR-10数据集

CIOWE10数据集包括60000个32×32彩色图像，分为10个类别，每个类有6000个图像。有50000个训练图像和10000个测试图像。
数据集分为五个训练批次和一个测试批次，每一个批次都有10000个图像。测试批次包含来自每个类别的随机抽取的1000个图像。训练批次在剩余图像中随机抽取，训练批次可能包含某一类的图像多于另一类。训练批次包含来自每个类的5000个图像。
下面是数据集中的类别以及10个随机抽取的图像：
![](https://github.com/freefrog1986/Articles/blob/master/%E5%AE%9E%E7%8E%B0Lenet-5%E6%A8%A1%E5%9E%8B%E5%AF%B9CIFAR-10%E6%95%B0%E6%8D%AE%E9%9B%86%E8%BF%9B%E8%A1%8C%E5%88%86%E7%B1%BB/cifar-10.png?raw=true)

## 目标

本文的目标是搭建LeNet模型，并在CIFAR-10数据集上进行训练和测试。使用python语言和tensorflow库。为了方便处理数据集，请下载helper.py文件并放在

- python 版本： Python '3.5.4'
- tensorflow 版本：tensorflow '1.1.0'

## 获取数据

数据集下载地址：[CIFAR-10 dataset for python](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)
请下载并解压缩。

## 预处理函数

***归一化***

我们知道，图像大小为[32,32,3]，其中第三维代表RGB通道，每一个通道的取值范围是[0,255]，所以我们将每一个像素数据除以255，相当于对数据进行归一化处理，得到归一化之后的数据取值范围是[0, 1]。

In [19]:
def normalize(x):
    """
    归一化至范围[0, 1]
    x: 像素数据列表
    return: 归一化后的数据
    """
    return x/255 

In [20]:
import numpy as np
normalize(np.array([120,329,255])) #测试

array([0.47058824, 1.29019608, 1.        ])

***One-hot编码***

这里需要对标签进行one-hot编码，首先创建一个与类别数量相同的全0数组，然后将对应位置置为1。例如，编码前的标签是4，编码后为[0,0,0,1,0,0,0,0,0,0]

In [21]:
def one_hot_encode(x):
    """
    one-hot编码.
    : x: 标签列表
    : return: Numpy array格式的one-hot编码标签列表
    """
    output = np.zeros((len(x),10)) # 根据输入数组的长度和类别数量创建数组
    for i in range(len(x)):
        output[i][x[i]] = 1 # 将数组的对应位置置为1 
    return output

In [25]:
one_hot_encode(np.array([1,3,6,2,4,9])) #测试

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

## 对所有数据进行预处理并保存

In [27]:
def preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode):
    """
    Preprocess Training and Validation Data
    """
    n_batches = 5
    valid_features = []
    valid_labels = []

    for batch_i in range(1, n_batches + 1):
        features, labels = load_cfar10_batch(cifar10_dataset_folder_path, batch_i)
        validation_count = int(len(features) * 0.1)

        # Prprocess and save a batch of training data
        _preprocess_and_save(
            normalize,
            one_hot_encode,
            features[:-validation_count],
            labels[:-validation_count],
            'preprocess_batch_' + str(batch_i) + '.p')

        # Use a portion of training batch for validation
        valid_features.extend(features[-validation_count:])
        valid_labels.extend(labels[-validation_count:])

    # Preprocess and Save all validation data
    _preprocess_and_save(
        normalize,
        one_hot_encode,
        np.array(valid_features),
        np.array(valid_labels),
        'preprocess_validation.p')

    with open(cifar10_dataset_folder_path + '/test_batch', mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')

    # load the training data
    test_features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    test_labels = batch['labels']

    # Preprocess and Save all training data
    _preprocess_and_save(
        normalize,
        one_hot_encode,
        np.array(test_features),
        np.array(test_labels),
        'preprocess_training.p')

In [29]:
# Preprocess Training, Validation, and Testing Data
import help
preprocess_and_save_data('./cifar-10-batches-py', normalize, one_hot_encode)

NameError: name 'load_cfar10_batch' is not defined